# Getting Raw Data from database

In [2]:
import mysql.connector

print("Creating Connection . . .")
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="aingg",
    database="35mmc"
)
curr = db.cursor()
print("Connceted")

# Getting all rows
print("Getting Data . . .")
curr.execute("SELECT * FROM 35mmc_raw_bk")

# fetch all
rawResult = curr.fetchall()
# close connection
print("Closing Connection . . .")
db.close()
print("Disconnected")


Creating Connection . . .
Connceted
Getting Data . . .
Closing Connection . . .
Disconnected


In [2]:
# ID ,        Title,        URL,          RawHTML
rawResult[0][3], rawResult[0][0], rawResult[0][1], rawResult[0][2][:100]

(1,
 'Panorama Wide Pic Toy Camera Mini-Review – by Holly Gilman',
 'https://www.35mmc.com/26/02/2021/panorama-wide-pic-review/',
 '\n\n \n\nPoint & Shoot\nPanorama Wide Pic Toy Camera Mini-Review – by Holly Gilman\nFebruary 26, 2021\n\n\nLa')

# Function for cleaning - extracting - stemming - tokenizing 

In [4]:
import re
import spacy
from nltk.corpus import stopwords

class TextCleaner:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")
        self.stop_words = set(stopwords.words('english'))

    def normalize(self, raw_text):
        """Remove special characters and lowercase text"""
        return re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", raw_text.lower())

    def remove_stopwords(self, raw_text):
        """Remove stopwords"""
        words = [word for word in raw_text.split() if word not in self.stop_words]
        return " ".join(words)

    def lemmatize(self, raw_text):
        """Perform lemmatization"""
        doc = self.nlp(raw_text)
        return [token.lemma_ for token in doc]

    def clean(self, raw_text):
        """Clean text by normalizing, removing stopwords, and lemmatizing"""
        raw_text = self.normalize(raw_text)
        raw_text = self.remove_stopwords(raw_text)
        return self.lemmatize(raw_text)

text_cleaner = TextCleaner()

normalized_text = text_cleaner.normalize(raw_text)

no_stopwords_text = text_cleaner.remove_stopwords(normalized_text)

lemmatized_text = text_cleaner.lemmatize(no_stopwords_text)

# Data Frame for ID and URLs

In [5]:
# Create New dataframe for web ID and URLs
import pandas as pd

webData = pd.DataFrame({'ID':[], 'URL':[]})
webData['ID'] = webData['ID'].astype(int)
webData['URL'] = webData['URL'].astype(str)
webData

,ID,URL


In [6]:
for i in range(len(rawResult)):
    new_row = pd.DataFrame({'ID':[rawResult[i][3]], 'URL':[rawResult[i][1]]})
    webData = pd.concat([webData, new_row])
webData    

,ID,URL
0,1,https://www.35mmc.com/26/02/2021/panorama-wide...
0,2,https://www.35mmc.com/20/05/2016/halina-af700-...
0,3,https://www.35mmc.com/15/08/2022/ricoh-mirai-o...
0,4,https://www.35mmc.com/28/10/2020/pf-micro-110-...
0,5,https://www.35mmc.com/05/06/2020/fujifilm-zoom...
...,...,...
0,1089,https://www.35mmc.com/27/06/2018/olympus-mju-i...
0,1090,https://www.35mmc.com/16/12/2018/konica-z-up-1...
0,1091,https://www.35mmc.com/10/07/2015/konica-mermai...
0,1092,https://www.35mmc.com/14/12/2020/pentax-espio-...


# Create Dataframe for Inverted Indexing

# Buggy Version

from collections import Counter
from tqdm import tqdm
import pandas as pd

invertedIndex = pd.DataFrame({'Gram':[], 'DocsFreq':[], 'DocsID_Dict':[]})
invertedIndex['DocsFreq'] = invertedIndex['DocsFreq'].astype(int)

# for i in tqdm(range(3)):
for i in tqdm(range(len(rawResult))):
    
    # get cleaned token
    text_cleaner = TextCleaner()
    normalized_text = text_cleaner.normalize(rawResult[i][2])
    no_stopwords_text = text_cleaner.remove_stopwords(normalized_text)
    lemmatized_text = text_cleaner.lemmatize(no_stopwords_text)
    
    # get token frequency
    freq_dict = Counter(lemmatized_text)
    
    for word in freq_dict:
        # If token already exists in dataframe
        if len(invertedIndex.loc[invertedIndex['Gram'] == word]) == 1:
            invertedIndex.loc[invertedIndex['Gram'] == word]['DocsID_Dict'][0][i] = freq_dict[word]
            invertedIndex.loc[invertedIndex['Gram'] == word, 'DocsFreq'] += 1
        
        # New Words
        else:
            test_gram = word
            test_DocsFreq = 1
            test_DocsID_Dict = {
                rawResult[i][3] : freq_dict[word]
            }
            new_row = pd.DataFrame({'Gram':[test_gram], 'DocsFreq':[test_DocsFreq], 'DocsID_Dict':[test_DocsID_Dict]})
            invertedIndex = pd.concat([invertedIndex, new_row])

In [1]:
from collections import Counter
from tqdm import tqdm
import pandas as pd

invertedIndex = pd.DataFrame({'Gram':[], 'DocsFreq':[], 'DocsID_Dict':[]})
invertedIndex['DocsFreq'] = invertedIndex['DocsFreq'].astype(int)

for i in tqdm(range(10)):
# for i in tqdm(range(len(rawResult))):
    
    # get cleaned token
    text_cleaner = TextCleaner()
    normalized_text = text_cleaner.normalize(rawResult[i][2])
    no_stopwords_text = text_cleaner.remove_stopwords(normalized_text)
    lemmatized_text = text_cleaner.lemmatize(no_stopwords_text)
    
    # get token frequency
    freq_dict = Counter(lemmatized_text)
    
    for word in freq_dict:
        # If token already exists in dataframe
        if len(invertedIndex.loc[invertedIndex['Gram'] == word]) == 1:
            invertedIndex.loc[invertedIndex['Gram'] == word]['DocsID_Dict'][0][rawResult[i][3]] = freq_dict[word]
            invertedIndex.loc[invertedIndex['Gram'] == word, 'DocsFreq'] += 1
        
        # New Words
        else:
            test_gram = word
            test_DocsFreq = 1
            test_DocsID_Dict = {
                rawResult[i][3] : freq_dict[word]
            }
            new_row = pd.DataFrame({'Gram':[test_gram], 'DocsFreq':[test_DocsFreq], 'DocsID_Dict':[test_DocsID_Dict]})
            invertedIndex = pd.concat([invertedIndex, new_row])

invertedIndex

  0%|          | 0/10 [00:00<?, ?it/s]


NameError: name 'TextCleaner' is not defined

In [ ]:
from collections import Counter
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd

invertedIndex = pd.DataFrame({'Gram':[], 'DocsFreq':[], 'DocsID_Dict':[]})
invertedIndex['DocsFreq'] = invertedIndex['DocsFreq'].astype(int)

# function to process the tasks in parallel
def process_task(rawResult_i):
    text_cleaner = TextCleaner()
    normalized_text = text_cleaner.normalize(rawResult_i[2])
    no_stopwords_text = text_cleaner.remove_stopwords(normalized_text)
    lemmatized_text = text_cleaner.lemmatize(no_stopwords_text)
    freq_dict = Counter(lemmatized_text)
    for word in freq_dict:
        # If token already exists in dataframe
        if len(invertedIndex.loc[invertedIndex['Gram'] == word]) == 1:
            invertedIndex.loc[invertedIndex['Gram'] == word]['DocsID_Dict'][0][rawResult_i[3]] = freq_dict[word]
            invertedIndex.loc[invertedIndex['Gram'] == word, 'DocsFreq'] += 1
        
        # New Words
        else:
            test_gram = word
            test_DocsFreq = 1
            test_DocsID_Dict = {
                rawResult_i[3] : freq_dict[word]
            }
            new_row = pd.DataFrame({'Gram':[test_gram], 'DocsFreq':[test_DocsFreq], 'DocsID_Dict':[test_DocsID_Dict]})
            invertedIndex = pd.concat([invertedIndex, new_row])
    return invertedIndex

# create a thread pool with 4 worker threads
with ThreadPoolExecutor() as executor:
    # submit the tasks to the thread pool
    futures = [executor.submit(process_task, rawResult[i]) for i in range(len(rawResult))]
    # wait for all tasks to complete
    for f in tqdm(as_completed(futures), total=len(rawResult)):
        f.result()


# ID ,        Title,        URL,          RawHTML
result[0][3], result[0][0], result[0][1], result[0][2][:100]

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
invertedIndex

In [23]:
from pathlib import Path  

filepath = Path('webData.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
webData.to_csv(filepath)  

filepath = Path('invertedIndex.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
invertedIndex.to_csv(filepath)  

In [29]:
safe_data = invertedIndex